In [7]:
! pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.8/124.9 MB 2.4 MB/s eta 0:00:52
   ---------------------------------------- 1.3/124.9 MB 2.6 MB/s eta 0:00:48
    --------------------------------------- 1.8/124.9 MB 2.8 MB/s eta 0:00:45
    --------------------------------------- 2.4/124.9 MB 2.5 MB/s eta 0:00:49
   - -------------------------------------- 3.1/124.9 MB 2.6 MB/s eta 0:00:47
   - -------------------------------------- 3.7/124.9 MB 2.7 MB/s eta 0:00:45
   - -------------------------------------- 4.5/124.9 MB 2.9 MB/s eta 0:00:43
   - -------------------------------------- 5.2/124.9 MB 2.9 MB/s eta 0:00:42
   - -------------------------------------- 6.0/124.9 MB 3.0 MB/s eta 0:00:40
   -- ------------------------------------- 6.6/124.9 MB 3.0 MB/s eta 0:00:40
   -- -

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from xgboost import XGBRegressor
import warnings

In [10]:
# importing the data
df=pd.read_csv('data/stud.csv')

In [11]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Preparing X and Y variable

In [15]:
x=df.drop(columns=['math_score'],axis=1)

In [16]:
x.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [17]:
y=df['math_score']

In [32]:
#create column transform
num_features = x.select_dtypes(include='int').columns
cat_features=x.select_dtypes(exclude='int').columns

In [49]:
num_features,cat_features

(Index(['reading_score', 'writing_score'], dtype='object'),
 Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
        'test_preparation_course'],
       dtype='object'))

In [34]:
cat_transformer=OneHotEncoder()
num_transformer=StandardScaler()

In [35]:
processing=ColumnTransformer(
    [
        ("OneHotEncoder",cat_transformer,cat_features),
        ("StandardScaler",num_transformer,num_features)
    ]
)

In [36]:
X=processing.fit_transform(x)

In [39]:
X.shape

(1000, 19)

In [40]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [42]:
X_train.shape,X_test.shape

((800, 19), (200, 19))

Creating an eveluating function 

In [64]:
def evaluate_model(true,predict):
    mae=mean_absolute_error(true,predict)
    mse=mean_squared_error(true,predict)
    rmse=np.sqrt(mse)
    r_score=r2_score(true,predict)
    return mae,r_score,rmse

In [65]:
models= {
    'Linear Regression':LinearRegression(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'Random Forest Regressor': RandomForestRegressor(),
    'Lasso': Lasso(),
    'Ridge':Ridge(),
    'XGBRegressor': XGBRegressor(),
    'AdaBoost Regressor':AdaBoostRegressor()


}

model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #make predictions
    y_train_predict=model.predict(X_train)
    y_test_predict=model.predict(X_test)

    model_train_mae , model_train_r2 , model_train_rmse = evaluate_model(y_train,y_train_predict)
    model_test_mae,model_test_r2,model_test_rmse=evaluate_model(y_test,y_test_predict)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')



Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.2815
- Mean Absolute Error: 4.2276
- R2 Score: 0.8789
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.5720
- Mean Absolute Error: 4.3948
- R2 Score: 0.8630


Decision Tree Regressor
Model performance for Training set
- Root Mean Squared Error: 0.0500
- Mean Absolute Error: 0.0025
- R2 Score: 1.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 7.7958
- Mean Absolute Error: 6.2150
- R2 Score: 0.7318


Random Forest Regressor
Model performance for Training set
- Root Mean Squared Error: 2.3179
- Mean Absolute Error: 1.8302
- R2 Score: 0.9767
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.6687
- Mean Absolute Error: 4.5865
- R2 Score: 0.8582


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.6102
- Mean Absolute Error: 5.1990
- R2 Score: 0.8104
---

In [75]:
result=pd.DataFrame(list(zip(model_list,r2_list)),columns=['model_name','r2_score'])

In [76]:
result

,model_name,r2_score
0,Linear Regression,0.862990
1,Decision Tree Regressor,0.731798
2,Random Forest Regressor,0.858192
3,Lasso,0.801773
4,Ridge,0.863213
5,XGBRegressor,0.818506
6,AdaBoost Regressor,0.825274
